In [1]:
import csv
import struct
import datetime
BIN_FILENAME = "datalog0527.bin"
today = datetime.datetime.today()
time_stamp = str(today.year) + "_" + str(today.month) + "_" + str(today.day)
# 手書き用
#time_stamp = "2023_" + "5_" ""

In [2]:
with open(f"{BIN_FILENAME}", "rb") as f:
    log = f.read()
    size = len(log)
    quatmag_array = []
    accgyro_array = []
    gnss_array = []
    altimeter_array = []
    for i in range(0, size, 32):
        data = log[i:i+32]
        if data[0:4] == b'\xf0\x00\x00\x00':
                quatmag_array.append(data)
        elif data[0:4] == b'\xf1\x00\x00\x00':
                gnss_array.append(data)
        elif data[0:4] == b'\xf3\x00\x00\x00':
                altimeter_array.append(data)
        elif data[0:4] == b'\xf6\x00\x00\x00':
                accgyro_array.append(data)

# altimeter構造体
- uint8_t HEADER
- unsigned long program_time
- float temp
- long height_1
- long height_2
- long height_3
- float air_pressure_1
- float air_pressure_2

In [7]:
# newline=""を指定しないとwriterowで謎の改行がでてしまう
with open(f"altimeter_{time_stamp}.csv", "w", newline="") as f:
    writer = csv.writer(f)
    pandas_header = ["header", "program_time", "temp", "height1", "height2", "height3", "air_pressure1", "air_pressure2"]
    writer.writerow(pandas_header)
    for i in range(len(altimeter_array)):
        array = []
        array.append(int.from_bytes(altimeter_array[i][0:4], "little"))
        array.append(int.from_bytes(altimeter_array[i][4:8], "little"))
        array.append(round(struct.unpack('<f', altimeter_array[i][8:12])[0], 2))
        array.append(int.from_bytes(altimeter_array[i][12:16], "little"))
        array.append(int.from_bytes(altimeter_array[i][16:20], "little"))
        array.append(int.from_bytes(altimeter_array[i][20:24], "little"))
        array.append(struct.unpack('<f', altimeter_array[i][24:28])[0])
        array.append(struct.unpack('<f', altimeter_array[i][28:32])[0])
        writer.writerow(array)  

# QuatMag構造体
- uint8_t HEADER
- float quat_w
- float quat_x
- float quat_y
- float quat_z
- float mag_x
- float mag_y
- float mag_z

In [4]:
with open(f"quatmag_{time_stamp}.csv", 'w', newline="") as f:
    writer = csv.writer(f)
    pandas_header = ["header", "quatW", "quatX", "quatY", "quatZ", "magX", "magY", "magZ"]
    writer.writerow(pandas_header)
    for i in range(len(quatmag_array)):
        array = []
        array.append(int.from_bytes(quatmag_array[i][0:4], "little"))
        array.append(struct.unpack('<f', quatmag_array[i][4:8])[0])
        array.append(struct.unpack('<f', quatmag_array[i][8:12])[0])
        array.append(struct.unpack('<f', quatmag_array[i][12:16])[0])
        array.append(struct.unpack('<f', quatmag_array[i][16:20])[0])
        array.append(struct.unpack('<f', quatmag_array[i][20:24])[0])
        array.append(struct.unpack('<f', quatmag_array[i][24:28])[0])
        array.append(struct.unpack('<f', quatmag_array[i][28:32])[0])
        writer.writerow(array)

# AccGyro構造体
- uint8_t HEADER
- unsigned long program_time
- float acc_x
- float acc_y
- float acc_z
- float gyro_x
- float gyro_y
- float gyro_z

In [5]:
with open(f"accgyro_{time_stamp}.csv", 'w', newline="") as f:
    writer = csv.writer(f)
    pandas_header = ["header", "program_time", "accX", "accY", "accZ", "gyroX", "gyroY", "gyroZ"]
    writer.writerow(pandas_header)
    for i in range(len(accgyro_array)):
        array = []
        array.append(int.from_bytes(accgyro_array[i][0:4], "little"))
        array.append(int.from_bytes(accgyro_array[i][4:8], "little"))
        array.append(struct.unpack('<f', accgyro_array[i][8:12])[0])
        array.append(struct.unpack('<f', accgyro_array[i][12:16])[0])
        array.append(struct.unpack('<f', accgyro_array[i][16:20])[0])
        array.append(struct.unpack('<f', accgyro_array[i][20:24])[0])
        array.append(struct.unpack('<f', accgyro_array[i][24:28])[0])
        array.append(struct.unpack('<f', accgyro_array[i][28:32])[0])
        writer.writerow(array)

# GNSS構造体
- uint8_t HEADER
- unsigned long program_time
- uint8_t unixEpoch
- long latitude
- long longitude
- long altitude
- long ground_speed
- long heading

In [6]:
with open(f"gnss_{time_stamp}.csv", 'w', newline="") as f:
    writer = csv.writer(f)
    pandas_header = ["header", "program_time", "unixEpoch", "latitude", "longitude", "altitude", "groundSpeed", "heading"]
    writer.writerow(pandas_header)
    for i in range(len(gnss_array)):
        array = []
        array.append(int.from_bytes(gnss_array[i][0:4], "little"))
        array.append(int.from_bytes(gnss_array[i][4:8], "little"))
        array.append(int.from_bytes(gnss_array[i][8:12], "little"))
        
        # u-bloxで取得した緯度経度は10^7のlongで表されるので、10^(-7)倍にした後に小数点を丸め込む
        array.append(round(int.from_bytes(gnss_array[i][12:16], "little")* 10**(-7),7))
        array.append(round(int.from_bytes(gnss_array[i][16:20], "little")* 10**(-7), 7))
        
        array.append(int.from_bytes(gnss_array[i][20:24], "little"))
        array.append(int.from_bytes(gnss_array[i][24:28], "little"))
        array.append(int.from_bytes(gnss_array[i][28:32], "little"))
        writer.writerow(array)